In [1]:
#[8] Certidão Negativa do Imobiliário:

In [2]:
#Bibliotecas:

#Deprecated:
#from selenium.webdriver import Chrome

#New Version: Selenium 4.0 Beta 1
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
 
#Options:
from selenium.webdriver.chrome.options import Options

#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument('-headless')

from selenium.webdriver.support.ui import WebDriverWait

#Exceptions:
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from random import randrange       #random number (used in secs to wait - time)
import time                        #wait to run
from datetime import datetime      #Data/Hora de hoje
import getpass                     #usr info
import pandas as pd
import json
import os
import glob
import shutil

In [3]:
#Configurações Impressão em .pdf:
caminho_download = 'C:\\Users\\felip\\Downloads'
chrome_options = Options()
settings = {
       "recentDestinations": [{
            "id": "Save as PDF",
            "origin": "local",
            "account": "",
        }],
        "selectedDestinationId": "Save as PDF",
        "version": 2
    }
exclude_default_swithes = ["disable-client-side-phishing-detection",
    "safebrowsing-disable-auto-update",
    "safebrowsing-disable-download-protection",
    "ignore-certificate-errors",]
prefs = {"plugins.always_open_pdf_externally": True,
        "download.default_directory": caminho_download,       
        "printing.print_preview_sticky_settings.appState": json.dumps(settings),
        "savefile.default_directory": 'C:\\Users\\felip\\Downloads'}     #ADD
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument('--kiosk-printing') #ADD
chrome_options.add_experimental_option("excludeSwitches", exclude_default_swithes)
chrome_options.add_experimental_option("useAutomationExtension", False)

In [4]:
#Cria uma pasta para a SC caso ela já não exista no sistema:
def criaPastaSC(codigo_SC): #Captura o código da SC
    newpath = r'C:\\Users\\felip\\OneDrive\\Área de Trabalho\\Documentacao\\Certidoes' 
    newpath = newpath + '\\' + codigo_SC
    if not os.path.exists(newpath):
        os.makedirs(newpath)

In [5]:
def moveCert(codigo_SC):
#Move a certidão baixada da pasta Downloads para a pasta da SC:

    #Lista os arquivos da Pasta Downloads:
    list_of_files = glob.glob(r'C:\Users\felip\Downloads\*.pdf') # * means all if need specific format then *.csv
    print(list_of_files)
    
    #Caso não exista nenhum .pdf pode dar erro!
    #Captura o último arquivo:
    latest_file = max(list_of_files, key=os.path.getctime)

    print(latest_file)
    
    #Renomeia a certidão baixada para: "Cert_Neg_Imob_SC" + index
     
        #Determina a posição do corte de string (Em "Download\\ no caminho da certidão baixada:")
    pos_fim_cut = latest_file.find("Downloads\\")
    pos_fim_cut = pos_fim_cut + 10
    print(pos_fim_cut)
        
        #Recorta o nome da certidão baixada:
    caminho_cert = latest_file[:pos_fim_cut]
    print(caminho_cert)
        
        #Renomeia o arquivo construindo o novo caminho à certidão:
            
            #Extrai a data e hora de hoje: (Adiciona-se como código ao final da certidão para evitar sobreposição)
    now = datetime.now()
    hoje = now.strftime("%d/%m/%Y %H:%M:%S") #dd/mm/yyyy hh:mm:ss
    hoje = str(hoje).replace("/","").replace(":","").replace(" ","")
    print(hoje)
    nv_caminho_cert = caminho_cert + "Cert_Neg_Imob_SC" + str(codigo_SC) + "_" + hoje + ".pdf"
    print(nv_caminho_cert)
    os.rename(latest_file, nv_caminho_cert)
    
    #Move o arquivo à pasta da SC:
    destino = "C:\\Users\\felip\\OneDrive\\Área de Trabalho\\Documentacao\\Certidoes\\"
    destino = destino + str(codigo_SC)
    shutil.move(nv_caminho_cert, destino)
    
    print(destino)

In [6]:
#Captura o nome do usuário:
usr = getpass.getuser()

In [7]:
#INPUTS:
df_cpf = pd.read_excel("C:/Users/felip/OneDrive/Área de Trabalho/Documentacao/Robos/INPUT/A_Consultar.xlsx", engine='openpyxl')

In [8]:
#Acesso ao Site:

#Link para Certidão Negativa do Imobiliário:
url = "https://portal.cidadao.topdata-info.com.br/praiagrande/certidao_imobiliario.php"

#Varre cada CLiente/Pedido do Input:
for index, row in df_cpf.iterrows():

    #Identifica se a certidão conseguiu ser extraída atribuindo True:
    sucesso = False
        
    while sucesso != True:
        
        print("Realizando SC" + str(index))
        #Dados do Cliente:
        num_lanc = str(df_cpf['Num_Lancamento'][index])
        #Tratamento da informação: Remove "." e "-" do Número do Lançamento:
        num_lanc2 = num_lanc.replace(".", "").replace("-", "")
        #Remove o dígito do número de lançamento:
        num_lanc3 = num_lanc2[:-1]
        
        #Inicializa o Chromedriver:
        #s = Service('chromedriver.exe')
        driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)
        
        #Página Alvo:
        driver.get(url)
        
        #CLica na mensagem de Cookies:
        time.sleep(1)
        elemento = (By.ID, "cookieAcceptBarConfirm")
        acesso = driver.find_element(*elemento).click()
        driver.refresh()
        
        #Preenche o Número do Lançamento:
        time.sleep(1)
        elemento = (By.XPATH, "/html/body/center/div[@id='geral']/div[@id='content']/div[@id='principal']/form/div[@id='formulario']/table[@class='box']/tbody/tr[@class='textoPadrao'][2]/td/input")
        acesso = driver.find_element(*elemento).send_keys(num_lanc3)
                
        #Clica no botão Pesquisar:
        time.sleep(1)
        elemento = (By.XPATH, "/html/body/center/div[@id='geral']/div[@id='content']/div[@id='principal']/form/div[@id='formulario']/table[@class='box']/tbody/tr[@class='textoPadrao'][4]/td/input[@class='botao']")
        acesso = driver.find_element(*elemento).click()  
        
        #Clica no botão Impirmir:
        time.sleep(1)
        elemento = (By.XPATH, "/html/body/center/div[@id='geral']/div[@id='content']/div[@id='principal']/form/div[@id='listagem_registro']/table[@class='box']/tbody/tr[5]/td/input[@class='botao']")
        acesso = driver.find_element(*elemento).click()
        
        #Cria pasta na rede para SC caso ela ainda não exista:
        nm_pasta = str(index) + "_rb8"
        criaPastaSC(str(index))
        
        #Move Certidão para a pasta da SC:
        time.sleep(2)
        #Pausa para que o Download seja completo antes de mover e renomear o arquivo da certidão:
        moveCert(str(index))
        sucesso = True        

Realizando SC0


C:\Users\felip\AppData\Local\Temp/ipykernel_54084/1457503195.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)


['C:\\Users\\felip\\Downloads\\doc (1).pdf', 'C:\\Users\\felip\\Downloads\\doc.pdf', 'C:\\Users\\felip\\Downloads\\Documento sem título (1).pdf', 'C:\\Users\\felip\\Downloads\\Documento sem título.pdf', 'C:\\Users\\felip\\Downloads\\doc_sujeira.pdf']
C:\Users\felip\Downloads\doc (1).pdf
25
C:\Users\felip\Downloads\
24122021230823
C:\Users\felip\Downloads\Cert_Neg_Imob_SC0_24122021230823.pdf
C:\Users\felip\OneDrive\Área de Trabalho\Documentacao\Certidoes\0
Realizando SC1
['C:\\Users\\felip\\Downloads\\doc (1).pdf', 'C:\\Users\\felip\\Downloads\\doc.pdf', 'C:\\Users\\felip\\Downloads\\Documento sem título (1).pdf', 'C:\\Users\\felip\\Downloads\\Documento sem título.pdf', 'C:\\Users\\felip\\Downloads\\doc_sujeira.pdf']
C:\Users\felip\Downloads\doc (1).pdf
25
C:\Users\felip\Downloads\
24122021230836
C:\Users\felip\Downloads\Cert_Neg_Imob_SC1_24122021230836.pdf
C:\Users\felip\OneDrive\Área de Trabalho\Documentacao\Certidoes\1
Realizando SC2
['C:\\Users\\felip\\Downloads\\doc (1).pdf', 'C:\\